## Import Modules

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from os import listdir
from os.path import isfile, join
import time
import math
from IPython.display import clear_output
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import torchvision.transforms as T
#from torchsampler import ImbalancedDatasetSampler
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print('cuda available: '+ str(torch.cuda.is_available()))
#from skimage.morphology import disk, binary_dilation

from models_avm import NestedUNet
from loss_fun_avm import compute_per_channel_dice, DiceLoss, FocalLoss
from tra_val_avm import train, validation
from data_loader_avm import Dataset

cuda available: True


## Data Path

In [2]:
path_tra1 = '/work/samhong833/Data_AVM/forUNetpp/1_tra/1'
list_tra1 = os.listdir(path_tra1)
for i in range(len(list_tra1)):
    list_tra1[i] = path_tra1+'/'+list_tra1[i]
    
list_tra = list_tra1
    
path_val1 = '/work/samhong833/Data_AVM/forUNetpp/2_val/1'
list_val1 = os.listdir(path_val1)
for i in range(len(list_val1)):
    list_val1[i] = path_val1+'/'+list_val1[i]
    
list_val = list_val1

path_ts1 = '/work/samhong833/Data_AVM/forUNetpp/3_ts/1'
list_ts1 = os.listdir(path_ts1)
for i in range(len(list_ts1)):
    list_ts1[i] = path_ts1+'/'+list_ts1[i]
    
list_ts = list_ts1

path_tra_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/1_tra'
path_val_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/2_val'
path_ts_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/3_ts'

## Data Loader

In [3]:
train_data = Dataset(list_tra,path_tra_lab_txt,rand_dilate=True,max_dilate_factor=30)
train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size = 12,
    shuffle = True,   
)

val_data = Dataset(list_val,path_val_lab_txt,rand_dilate=True,max_dilate_factor=30)
val_loader = torch.utils.data.DataLoader(
    dataset = val_data,           
    batch_size = 4,                 
    shuffle = False,              
)

## Initialize Model and Optimization Parameters

In [4]:
# call model cuda for gpu
model = NestedUNet().cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
kwargs = {"alpha": 0.5, "gamma": 3, "reduction": 'mean'}
criterion_FL = FocalLoss(**kwargs)
criterion_DICE = DiceLoss()
loss = [criterion_FL,criterion_DICE]

## Start to Train

In [ ]:
# Create Directory
path = '/work/samhong833/Models/Seg_dia30'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

# Train the Model
epochs = 500 # The number of epochs

valloss = 0
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(path=path,
          model=model,
          loss=loss,
          optimizer=optimizer,
          dataloader=train_loader,
          epoch=epoch,
          scheduler=scheduler)
    print('-' * 89)
    vallossnew = validation(path=path,
          model=model,
          loss=loss,
          dataloader=val_loader,
          epoch=epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    scheduler.step()
    
    f2 = open(path + '/model_info.txt', 'a')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')        
    f2.close()
    

| epoch   1 |   100/  290 batches | lr 0.0001000 | ms/batch 107766.67 | loss_fl 0.004183 | loss_dice  0.25 | 
| epoch   1 |   200/  290 batches | lr 0.0001000 | ms/batch 105722.35 | loss_fl 0.002860 | loss_dice  0.23 | 
| epoch   1 |   290/  290 batches | lr 0.0001000 | ms/batch 94354.56 | loss_fl 0.002308 | loss_dice  0.21 | 
-----------------------------------------------------------------------------------------
| epoch   1 |   100/  118 batches | | ms/batch 16860.92 | val_loss_fl 0.001188 | val_loss_dice  0.19 | 
| epoch   1 |   118/  118 batches | | ms/batch 3013.97 | val_loss_fl 0.001179 | val_loss_dice  0.18 | 
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 327.72s
-----------------------------------------------------------------------------------------
| epoch   2 |   100/  290 batches | lr 0.0001000 | ms/batch 105745.32 | loss_fl 0.000850 | loss_dice  0.15 | 
| epoch   2 |   200/  290 batches | lr 0.0001000 

| epoch  11 |   100/  290 batches | lr 0.0000950 | ms/batch 104520.90 | loss_fl 0.000278 | loss_dice  0.10 | 
| epoch  11 |   200/  290 batches | lr 0.0000950 | ms/batch 104719.63 | loss_fl 0.000275 | loss_dice  0.10 | 
| epoch  11 |   290/  290 batches | lr 0.0000950 | ms/batch 93952.52 | loss_fl 0.000275 | loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| epoch  11 |   100/  118 batches | | ms/batch 16885.74 | val_loss_fl 0.000696 | val_loss_dice  0.13 | 
| epoch  11 |   118/  118 batches | | ms/batch 3016.71 | val_loss_fl 0.000686 | val_loss_dice  0.13 | 
-----------------------------------------------------------------------------------------
| end of epoch  11 | time: 323.10s
-----------------------------------------------------------------------------------------
| epoch  12 |   100/  290 batches | lr 0.0000950 | ms/batch 104666.39 | loss_fl 0.000267 | loss_dice  0.10 | 
| epoch  12 |   200/  290 batches | lr 0.0000950 

| epoch  21 |   100/  290 batches | lr 0.0000902 | ms/batch 105150.20 | loss_fl 0.000201 | loss_dice  0.08 | 
| epoch  21 |   200/  290 batches | lr 0.0000902 | ms/batch 105169.79 | loss_fl 0.000194 | loss_dice  0.08 | 
| epoch  21 |   290/  290 batches | lr 0.0000902 | ms/batch 94616.14 | loss_fl 0.000197 | loss_dice  0.08 | 
-----------------------------------------------------------------------------------------
| epoch  21 |   100/  118 batches | | ms/batch 17219.29 | val_loss_fl 0.000626 | val_loss_dice  0.12 | 
| epoch  21 |   118/  118 batches | | ms/batch 3052.72 | val_loss_fl 0.000614 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch  21 | time: 325.21s
-----------------------------------------------------------------------------------------
| epoch  22 |   100/  290 batches | lr 0.0000902 | ms/batch 105260.78 | loss_fl 0.000197 | loss_dice  0.08 | 
| epoch  22 |   200/  290 batches | lr 0.0000902 

| epoch  31 |   100/  290 batches | lr 0.0000857 | ms/batch 105130.93 | loss_fl 0.000139 | loss_dice  0.06 | 
| epoch  31 |   200/  290 batches | lr 0.0000857 | ms/batch 105679.65 | loss_fl 0.000139 | loss_dice  0.06 | 
| epoch  31 |   290/  290 batches | lr 0.0000857 | ms/batch 94226.16 | loss_fl 0.000141 | loss_dice  0.06 | 
-----------------------------------------------------------------------------------------
| epoch  31 |   100/  118 batches | | ms/batch 16950.28 | val_loss_fl 0.000798 | val_loss_dice  0.11 | 
| epoch  31 |   118/  118 batches | | ms/batch 3043.88 | val_loss_fl 0.000778 | val_loss_dice  0.10 | 
-----------------------------------------------------------------------------------------
| end of epoch  31 | time: 325.04s
-----------------------------------------------------------------------------------------
| epoch  32 |   100/  290 batches | lr 0.0000857 | ms/batch 105252.23 | loss_fl 0.000135 | loss_dice  0.06 | 
| epoch  32 |   200/  290 batches | lr 0.0000857 

| epoch  41 |   100/  290 batches | lr 0.0000815 | ms/batch 105221.27 | loss_fl 0.000110 | loss_dice  0.05 | 
| epoch  41 |   200/  290 batches | lr 0.0000815 | ms/batch 105499.81 | loss_fl 0.000108 | loss_dice  0.05 | 
| epoch  41 |   290/  290 batches | lr 0.0000815 | ms/batch 95044.98 | loss_fl 0.000107 | loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| epoch  41 |   100/  118 batches | | ms/batch 16990.63 | val_loss_fl 0.001044 | val_loss_dice  0.11 | 
| epoch  41 |   118/  118 batches | | ms/batch 3045.83 | val_loss_fl 0.001027 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch  41 | time: 325.81s
-----------------------------------------------------------------------------------------
| epoch  42 |   100/  290 batches | lr 0.0000815 | ms/batch 105146.71 | loss_fl 0.000103 | loss_dice  0.05 | 
| epoch  42 |   200/  290 batches | lr 0.0000815 

...

| epoch 171 |   100/  290 batches | lr 0.0000418 | ms/batch 110390.14 | loss_fl 0.000022 | loss_dice  0.01 | 
| epoch 171 |   200/  290 batches | lr 0.0000418 | ms/batch 112149.93 | loss_fl 0.000021 | loss_dice  0.01 | 
| epoch 171 |   290/  290 batches | lr 0.0000418 | ms/batch 101656.07 | loss_fl 0.000021 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 171 |   100/  118 batches | | ms/batch 23409.84 | val_loss_fl 0.003166 | val_loss_dice  0.11 | 
| epoch 171 |   118/  118 batches | | ms/batch 3966.30 | val_loss_fl 0.003162 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 171 | time: 351.58s
-----------------------------------------------------------------------------------------
| epoch 172 |   100/  290 batches | lr 0.0000418 | ms/batch 115336.14 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 172 |   200/  290 batches | lr 0.0000418

| epoch 181 |   100/  290 batches | lr 0.0000397 | ms/batch 107052.79 | loss_fl 0.000019 | loss_dice  0.01 | 
| epoch 181 |   200/  290 batches | lr 0.0000397 | ms/batch 108996.67 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 181 |   290/  290 batches | lr 0.0000397 | ms/batch 101667.75 | loss_fl 0.000018 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 181 |   100/  118 batches | | ms/batch 21044.83 | val_loss_fl 0.003537 | val_loss_dice  0.11 | 
| epoch 181 |   118/  118 batches | | ms/batch 3866.10 | val_loss_fl 0.003517 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 181 | time: 342.63s
-----------------------------------------------------------------------------------------
| epoch 182 |   100/  290 batches | lr 0.0000397 | ms/batch 109194.72 | loss_fl 0.000018 | loss_dice  0.01 | 
| epoch 182 |   200/  290 batches | lr 0.0000397

| epoch 191 |   100/  290 batches | lr 0.0000377 | ms/batch 109476.25 | loss_fl 0.000017 | loss_dice  0.01 | 
| epoch 191 |   200/  290 batches | lr 0.0000377 | ms/batch 111413.96 | loss_fl 0.000017 | loss_dice  0.01 | 
| epoch 191 |   290/  290 batches | lr 0.0000377 | ms/batch 102899.52 | loss_fl 0.000017 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 191 |   100/  118 batches | | ms/batch 19181.30 | val_loss_fl 0.003833 | val_loss_dice  0.12 | 
| epoch 191 |   118/  118 batches | | ms/batch 3729.00 | val_loss_fl 0.003806 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 191 | time: 346.70s
-----------------------------------------------------------------------------------------
| epoch 192 |   100/  290 batches | lr 0.0000377 | ms/batch 110119.26 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 192 |   200/  290 batches | lr 0.0000377

| epoch 201 |   100/  290 batches | lr 0.0000358 | ms/batch 107983.43 | loss_fl 0.000015 | loss_dice  0.01 | 
| epoch 201 |   200/  290 batches | lr 0.0000358 | ms/batch 107561.15 | loss_fl 0.000016 | loss_dice  0.01 | 
| epoch 201 |   290/  290 batches | lr 0.0000358 | ms/batch 96719.62 | loss_fl 0.000016 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 201 |   100/  118 batches | | ms/batch 17136.39 | val_loss_fl 0.003798 | val_loss_dice  0.12 | 
| epoch 201 |   118/  118 batches | | ms/batch 3085.09 | val_loss_fl 0.003781 | val_loss_dice  0.12 | 
-----------------------------------------------------------------------------------------
| end of epoch 201 | time: 332.49s
-----------------------------------------------------------------------------------------
| epoch 202 |   100/  290 batches | lr 0.0000358 | ms/batch 106813.03 | loss_fl 0.000015 | loss_dice  0.01 | 
| epoch 202 |   200/  290 batches | lr 0.0000358 

| epoch 211 |   100/  290 batches | lr 0.0000341 | ms/batch 108054.78 | loss_fl 0.000014 | loss_dice  0.01 | 
| epoch 211 |   200/  290 batches | lr 0.0000341 | ms/batch 110328.52 | loss_fl 0.000014 | loss_dice  0.01 | 
| epoch 211 |   290/  290 batches | lr 0.0000341 | ms/batch 99491.31 | loss_fl 0.000014 | loss_dice  0.01 | 
-----------------------------------------------------------------------------------------
| epoch 211 |   100/  118 batches | | ms/batch 17142.07 | val_loss_fl 0.004183 | val_loss_dice  0.12 | 
| epoch 211 |   118/  118 batches | | ms/batch 3066.12 | val_loss_fl 0.004165 | val_loss_dice  0.11 | 
-----------------------------------------------------------------------------------------
| end of epoch 211 | time: 338.09s
-----------------------------------------------------------------------------------------
| epoch 212 |   100/  290 batches | lr 0.0000341 | ms/batch 110376.97 | loss_fl 0.000015 | loss_dice  0.01 | 
| epoch 212 |   200/  290 batches | lr 0.0000341 

# Log ^